In [1]:
!git clone https://github.com/redaelkate/modeltest
!pip install kaggle

Cloning into 'modeltest'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 64 (delta 34), reused 22 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 52.20 KiB | 8.70 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
!cp -r /kaggle/input/damage-classification-models/weights /kaggle/working/modeltest


In [ ]:
from fastapi import FastAPI

from pydantic import BaseModel

import pickle

import json

import uvicorn

from pyngrok import ngrok

from fastapi.middleware.cors import CORSMiddleware

import nest_asyncio

In [4]:
!pip install fastapi

!pip install uvicorn

!pip install pickle5

!pip install pydantic

!pip install requests

!pip install pypi-json

!pip install pyngrok

!pip install nest-asyncio
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=125856 sha256=17f9208242db3accfcd46036b8a863b79fd882a678f0e8b0d1857e34e0006f88
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 9.9 MB/s eta 0:00:00


In [26]:
%cd /kaggle/working/modeltest

from fastapi import FastAPI, File, UploadFile,Request
from fastapi.responses import FileResponse, JSONResponse
import os
import torch
import torch.nn as nn
from predict34_loc import process_image_with_models
from predict50_loc import loc_50
from predict92_loc import loc_92
from predict154_loc import loc_154
from predict34cls import cls_34
from predict50cls import cls_50
from predict92cls import cls_92
from predict154cls import cls_154

from zoo.models import Res34_Unet_Loc
from zoo.models import  SeResNext50_Unet_Loc, Dpn92_Unet_Loc, SeNet154_Unet_Loc,Res34_Unet_Double,SeResNext50_Unet_Double,Dpn92_Unet_Double,SeNet154_Unet_Double
from io import BytesIO
from pathlib import Path
from PIL import Image
import cv2
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
import shutil
import os
import torch.nn as nn
import torch.optim as optim
from torchvision.models import swin_t
import warnings
warnings.filterwarnings('ignore')
from PIL import Image 
from torchvision import transforms
from fastapi import FastAPI
from torchvision import transforms
import torch
import os

from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
import numpy as np
np.random.seed(1)
import random
random.seed(1)

import pandas as pd
from tqdm import tqdm
import timeit
import cv2

from skimage.morphology import remove_small_objects

import matplotlib.pyplot as plt
import seaborn as sns

from skimage.morphology import square, dilation
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

# pred_folders = ['dpn92cls_cce_0_tuned', 'dpn92cls_cce_1_tuned', 'dpn92cls_cce_2_tuned'] + ['res34cls2_0_tuned', 'res34cls2_1_tuned', 'res34cls2_2_tuned'] + ['res50cls_cce_0_tuned', 'res50cls_cce_1_tuned', 'res50cls_cce_2_tuned'] + ['se154cls_0_tuned', 'se154cls_1_tuned', 'se154cls_2_tuned']
# pred_coefs = [1.0] * 12
# loc_folders = ['pred50_loc_tuned', 'pred92_loc_tuned', 'pred34_loc', 'pred154_loc']
# loc_coefs = [1.0] * 4   

# sub_folder = '/kaggle/working/submission/'
# _thr = [0.38, 0.13, 0.14]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#submission file
pred_folders = ['/kaggle/working/dpn92cls_cce_0_tuned', '/kaggle/working/dpn92cls_cce_1_tuned', '/kaggle/working/dpn92cls_cce_2_tuned'] + ['/kaggle/working/res34cls2_0_tuned', '/kaggle/working/res34cls2_1_tuned', '/kaggle/working/res34cls2_2_tuned'] + ['/kaggle/working/res50cls_cce_0_tuned', '/kaggle/working/res50cls_cce_1_tuned', '/kaggle/working/res50cls_cce_2_tuned'] + ['/kaggle/working/se154cls_0_tuned', '/kaggle/working/se154cls_1_tuned', '/kaggle/working/se154cls_2_tuned']
pred_coefs = [1.0] * 12
loc_folders = ['/kaggle/working/pred50_loc_tuned', '/kaggle/working/pred92_loc_tuned', '/kaggle/working/pred34_loc', '/kaggle/working/pred154_loc']
loc_coefs = [1.0] * 4   
sub_folder = '/kaggle/working/submission'
_thr = [0.38, 0.13, 0.14]
    
def process_image(f):
    preds = []
    _i = -1
    for d in pred_folders:
        _i += 1
        msk1 = cv2.imread(path.join(d, f), cv2.IMREAD_UNCHANGED)
        msk2 = cv2.imread(path.join(d, f.replace('_part1', '_part2')), cv2.IMREAD_UNCHANGED)
        msk = np.concatenate([msk1, msk2[..., 1:]], axis=2)
        preds.append(msk * pred_coefs[_i])
    preds = np.asarray(preds).astype('float').sum(axis=0) / np.sum(pred_coefs) / 255
    
    loc_preds = []
    _i = -1
    for d in loc_folders:
        _i += 1
        msk = cv2.imread(path.join(d, f), cv2.IMREAD_UNCHANGED)
        loc_preds.append(msk * loc_coefs[_i])
    loc_preds = np.asarray(loc_preds).astype('float').sum(axis=0) / np.sum(loc_coefs) / 255

    msk_dmg = preds[..., 1:].argmax(axis=2) + 1
    msk_loc = (1 * ((loc_preds > _thr[0]) | ((loc_preds > _thr[1]) & (msk_dmg > 1) & (msk_dmg < 4)) | ((loc_preds > _thr[2]) & (msk_dmg > 1)))).astype('uint8')
    
    msk_dmg = msk_dmg * msk_loc
    _msk = (msk_dmg == 2)
    if _msk.sum() > 0:
        _msk = dilation(_msk, square(5))
        msk_dmg[_msk & msk_dmg == 1] = 2

    msk_dmg = msk_dmg.astype('uint8')
    cv2.imwrite(path.join(sub_folder, '{0}'.format(f.replace('_pre_', '_localization_').replace('_part1.png', '_prediction.png'))), msk_loc, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(path.join(sub_folder, '{0}'.format(f.replace('_pre_', '_damage_').replace('_part1.png', '_prediction.png'))), msk_dmg, [cv2.IMWRITE_PNG_COMPRESSION, 9])
def predict_level_ofdamage():
    all_files = []
    for f in tqdm(sorted(listdir(pred_folders[0]))):
        if '_part1.png' in f:
            all_files.append(f)
    print(all_files)
    for f in tqdm(all_files):
        try:
            process_image(f)
            print("files",f)
        except Exception as e:
            print(f"Error processing file {f}: {e}") 
    submission="/kaggle/working/submission/"
    for f in tqdm(sorted(listdir(submission))):
        dir=submission+f
        print(dir)
        if "damage" in f:
            damage_image=cv2.imread(dir,cv2.IMREAD_GRAYSCALE)
        else:
            localization_image=cv2.imread(dir,cv2.IMREAD_GRAYSCALE)
    L = localization_image 
    D = damage_image
    mask = (L == 1) & (D >= 1) & (D <= 4)
    localized_damage_values = D[mask]
    if len(localized_damage_values) > 0:
        average_damage = np.mean(localized_damage_values)
    else:
        average_damage = 0
    if average_damage > 0 and average_damage <= 1.15:
        predicted_damage_level = 1
    elif average_damage > 1.15 and average_damage <= 2.15:
        predicted_damage_level = 2
    elif average_damage > 2.15 and average_damage <= 3.15:
        predicted_damage_level = 3
    elif average_damage > 3.15 and average_damage <= 4:
        predicted_damage_level = 4
    else:
        predicted_damage_level = 0 
    return predicted_damage_level
    return predicted_damage_level
# Load swin model 
def create_model(num_classes=6):

    model = swin_t(weights='DEFAULT')
    
    # Modify the head for our number of classes
    num_features = model.head.in_features
    model.head = nn.Sequential(
        nn.LayerNorm(num_features),
        nn.Linear(num_features, 512),
        nn.GELU(),
        nn.Dropout(0.1),
        nn.Linear(512, num_classes)
    )
    
    return model.to(device)
checkpoint_path = "/kaggle/input/swinmodel_cls/pytorch/default/1/swin_disaster_classifier.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
classes = checkpoint['classes']
swin_model = create_model(num_classes=len(classes))
swin_model.load_state_dict(checkpoint['model_state_dict'])
def predict_image_class(image_path, model, classes):
    image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to match model input
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(           # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = image_transforms(image).unsqueeze(0)  # Add batch dimension
    
    # Move tensor to the same device as the model
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    
    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)  # Forward pass
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)  # Convert logits to probabilities
        predicted_idx = torch.argmax(probabilities).item()  # Get the index of the highest probability
    
    # Return predicted class and confidence
    predicted_class = classes[predicted_idx]
    return predicted_class
# Load loc 34 models
loc_34_models = []
for seed in [0, 1, 2]:
    snap_to_load = f'/kaggle/working/modeltest/weights/res34_loc_{seed}_1_best' 
    model = Res34_Unet_Loc(pretrained=None)
    model = nn.DataParallel(model).to(device)
    print(f"=> loading checkpoint '{snap_to_load}'")
    checkpoint = torch.load(snap_to_load, map_location=device)

    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
            sd[k] = loaded_dict[k]
    model.load_state_dict(sd)
    model.eval()
    loc_34_models.append(model)
# load loc 50 models 
loc_50_models = []
for seed in [0, 1, 2]:
    snap_to_load = f'/kaggle/working/modeltest/weights/res50_loc_{seed}_tuned_best'
    model = SeResNext50_Unet_Loc(pretrained=None)
    model = nn.DataParallel(model).to(device)
    print("=> loading checkpoint '{}'".format(snap_to_load))
    checkpoint = torch.load(snap_to_load, map_location=device)
    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
            sd[k] = loaded_dict[k]
    loaded_dict = sd
    model.load_state_dict(loaded_dict)
    print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
    model.eval()
    loc_50_models.append(model)

#load loc 92 models
loc_92_models = []

for seed in [0, 1, 2]:
    snap_to_load = f'/kaggle/working/modeltest/weights/dpn92_loc_{seed}_tuned_best'

    model = Dpn92_Unet_Loc(pretrained=None)
    model = nn.DataParallel(model).to(device)
    print("=> loading checkpoint '{}'".format(snap_to_load))
    checkpoint = torch.load(snap_to_load, map_location=device)
    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
            sd[k] = loaded_dict[k]
    loaded_dict = sd
    model.load_state_dict(loaded_dict)
    print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
    model.eval()
    loc_92_models.append(model)
#load 154 loc models
loc_154_models = []
for seed in [0, 1, 2]:
    snap_to_load = f'/kaggle/working/modeltest/weights/se154_loc_{seed}_1_best'
    model = SeNet154_Unet_Loc(pretrained=None)
    model = nn.DataParallel(model).to(device)
    print("=> loading checkpoint '{}'".format(snap_to_load))
    checkpoint = torch.load(snap_to_load, map_location=device)
    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
            sd[k] = loaded_dict[k]
    loaded_dict = sd
    model.load_state_dict(loaded_dict)
    print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
    model.eval()
    loc_154_models.append(model)
#load 34 cls models
#'res34cls2_0_tuned', 'res34cls2_1_tuned', 'res34cls2_2_tuned'
models_res34_0 = []
snap_to_load = '/kaggle/working/modeltest/weights/res34_cls2_0_tuned_best'
model = Res34_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_res34_0.append(model)
models_res34_1 = []
snap_to_load = '/kaggle/working/modeltest/weights/res34_cls2_1_tuned_best'
model = Res34_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_res34_1.append(model)
models_res34_2 = []
snap_to_load = '/kaggle/working/modeltest/weights/res34_cls2_2_tuned_best'
model = Res34_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_res34_2.append(model)
# cls 50 
models_50_0=[]
snap_to_load = '/kaggle/working/modeltest/weights/res50_cls_cce_0_tuned_best'
model = SeResNext50_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
model.eval()
models_50_0.append(model)
models_50_1=[]
snap_to_load = '/kaggle/working/modeltest/weights/res50_cls_cce_1_tuned_best'
model = SeResNext50_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)  
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
model.eval()
models_50_1.append(model)


models_50_2=[]
snap_to_load = '/kaggle/working/modeltest/weights/res50_cls_cce_2_tuned_best'
model = SeResNext50_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
model.eval()
models_50_2.append(model)

##92_cls
models_92_0 = []
snap_to_load = '/kaggle/working/modeltest/weights/dpn92_cls_cce_0_tuned_best'
model = Dpn92_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_92_0.append(model)


models_92_1 = []
snap_to_load = '/kaggle/working/modeltest/weights/dpn92_cls_cce_1_tuned_best'
model = Dpn92_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_92_1.append(model)

models_92_2 = []
snap_to_load = '/kaggle/working/modeltest/weights/dpn92_cls_cce_2_tuned_best'
model = Dpn92_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_92_2.append(model)


## cls 154
models_154_0=[]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
snap_to_load = '/kaggle/working/modeltest/weights/se154_cls_cce_0_tuned_best'
model = SeNet154_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_154_0.append(model)

models_154_1=[]
snap_to_load = '/kaggle/working/modeltest/weights/se154_cls_cce_1_tuned_best'
model = SeNet154_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_154_1.append(model)


models_154_2=[]
snap_to_load = '/kaggle/working/modeltest/weights/se154_cls_cce_2_tuned_best'
model = SeNet154_Unet_Double(pretrained=None)
model = nn.DataParallel(model).to(device)
print("=> loading checkpoint '{}'".format(snap_to_load))
checkpoint = torch.load(snap_to_load, map_location=device)
loaded_dict = checkpoint['state_dict']
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)
print("loaded checkpoint '{}' (epoch {}, best_score {})".format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
model.eval()
models_154_2.append(model)


app = FastAPI()

@app.get("/")
def health_check():
    return {"status": "Running"}

# Echo endpoint
@app.post("/echo/")
async def echo_message(request: Request):
    body = await request.body()  # Read raw body
    return JSONResponse(content={"echoed_message": body.decode("utf-8")})
# Define your Kaggle working directory
mod_34_loc = "/kaggle/working/pred34_loc"
mod_50_loc="/kaggle/working/pred50_loc_tuned"
mod_92_loc="/kaggle/working/pred92_loc_tuned"
mod_154_loc="/kaggle/working/pred154_loc"
mod_34_cls_0="/kaggle/working/res34cls2_0_tuned"
mod_34_cls_1="/kaggle/working/res34cls2_1_tuned"
mod_34_cls_2="/kaggle/working/res34cls2_2_tuned"
mod_154_cls_0="/kaggle/working/se154cls_0_tuned"
mod_154_cls_1="/kaggle/working/se154cls_1_tuned"
mod_154_cls_2="/kaggle/working/se154cls_2_tuned"
mod_50_cls_0="/kaggle/working/res50cls_cce_0_tuned"
mod_50_cls_1="/kaggle/working/res50cls_cce_1_tuned"
mod_50_cls_2="/kaggle/working/res50cls_cce_2_tuned"
mod_92_cls_0="/kaggle/working/dpn92cls_cce_0_tuned"
mod_92_cls_1="/kaggle/working/dpn92cls_cce_1_tuned"
mod_92_cls_2="/kaggle/working/dpn92cls_cce_2_tuned"
# Directory to save the uploaded file temporarily
pre_file = "/kaggle/working/pre_file"
post_file="/kaggle/working/post_file"
# os.makedirs(temp_dir, exist_ok=True)

@app.post("/upload-image/")
async def upload_and_return_image( file1: UploadFile = File(...), file2: UploadFile = File(...)
):
    try:
        # Save the uploaded file to a temporary path
        pre_file_path = os.path.join(pre_file, file1.filename)
        with open(pre_file_path, "wb") as buffer:
            shutil.copyfileobj(file1.file, buffer)

        # Verify that the file was saved correctly
        if not os.path.exists(pre_file_path):
            return {"error": "File not saved correctly."}
        
        # Open the image to verify it is valid
        image_pre = Image.open(pre_file_path)
        
        print(f"Pre Image loaded successfully: {pre_file_path}, size: {image_pre.size}")
        post_file_path = os.path.join(post_file, file2.filename)
        with open(post_file_path, "wb") as buffer:
            shutil.copyfileobj(file2.file, buffer)

        # Verify that the file was saved correctly
        if not os.path.exists(post_file_path):
            return {"error": "File not saved correctly."}
        
        # Open the image to verify it is valid
        image_post = Image.open(post_file_path)
        folname1,ex=os.path.splitext(file1.filename)
        folname2,ex=os.path.splitext(file2.filename)
        print(f"Post Image loaded successfully: {post_file_path}, size: {image_post.size}")
        
        # Process the image
        msk = process_image_with_models(loc_34_models, image_pre)  # Ensure this function returns the correct shape and values
        
        # Debug: Check the shape and dtype of imgmask
        
        # Convert the mask to uint8
        # Ensure correct scaling
        
        #Define the output path
        output_filename = f"{os.path.basename(folname1)}_part1.png"
        output_path = os.path.join(mod_34_loc, output_filename)
        

        # Debug: Check the mask before saving
        if msk.ndim == 2:
            print("Mask is single-channel, adding dimensions...")
            msk = cv2.cvtColor(msk, cv2.COLOR_GRAY2BGR)  # Ensure it is 3-channel if needed

        # Save the processed mask as a PNG image
        cv2.imwrite(output_path, msk[..., 0], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        print(f"Processed image saved at: {output_path}")
        #loc50
        
        msk = loc_50(loc_50_models, image_pre)  # Ensure this function returns the correct shape and values

        output_filename = f"{os.path.basename(folname1)}_part1.png"
        output_path = os.path.join(mod_50_loc, output_filename)

        # Debug: Check the mask before saving
        if msk.ndim == 2:
            print("Mask is single-channel, adding dimensions...")
            msk = cv2.cvtColor(msk, cv2.COLOR_GRAY2BGR)  # Ensure it is 3-channel if needed

        # Save the processed mask as a PNG image
        cv2.imwrite(output_path, msk[..., 0], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        print(f"Processed image saved at: {output_path}")
        #loc 92
        msk = loc_92(loc_92_models, image_pre)  # Ensure this function returns the correct shape and values

        output_filename = f"{os.path.basename(folname1)}_part1.png"
        output_path = os.path.join(mod_92_loc, output_filename)

        # Debug: Check the mask before saving
        if msk.ndim == 2:
            print("Mask is single-channel, adding dimensions...")
            msk = cv2.cvtColor(msk, cv2.COLOR_GRAY2BGR)  # Ensure it is 3-channel if needed

        # Save the processed mask as a PNG image
        cv2.imwrite(output_path, msk[..., 0], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        # loc 154
        msk = loc_154(loc_154_models, image_pre)  # Ensure this function returns the correct shape and values

        output_filename = f"{os.path.basename(folname1)}_part1.png"
        output_path = os.path.join(mod_154_loc, output_filename)

        # Debug: Check the mask before saving
        if msk.ndim == 2:
            print("Mask is single-channel, adding dimensions...")
            msk = cv2.cvtColor(msk, cv2.COLOR_GRAY2BGR)  # Ensure it is 3-channel if needed

        # Save the processed mask as a PNG image
        cv2.imwrite(output_path, msk[..., 0], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        print(f"Processed image saved at: {output_path}")
        #csl34_0
        msk=cls_34(models_res34_0,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_34_cls_0, output_filename_1)
        output_path_2 = os.path.join(mod_34_cls_0, output_filename_2)
        
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        #csl34_1
        msk=cls_34(models_res34_1,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_34_cls_1, output_filename_1)
        output_path_2 = os.path.join(mod_34_cls_1, output_filename_2)
        
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        #csl34_2
        msk=cls_34(models_res34_2,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_34_cls_2, output_filename_1)
        output_path_2 = os.path.join(mod_34_cls_2, output_filename_2)
        
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        #cls50
        msk=cls_50(models_50_0,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_50_cls_0, output_filename_1)
        output_path_2 = os.path.join(mod_50_cls_0, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        
        msk=cls_50(models_50_1,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_50_cls_1, output_filename_1)
        output_path_2 = os.path.join(mod_50_cls_1, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        msk=cls_50(models_50_2,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_50_cls_2, output_filename_1)
        output_path_2 = os.path.join(mod_50_cls_2, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        #cls92

        msk=cls_92(models_92_0,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_92_cls_0, output_filename_1)
        output_path_2 = os.path.join(mod_92_cls_0, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        msk=cls_92(models_92_1,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_92_cls_1, output_filename_1)
        output_path_2 = os.path.join(mod_92_cls_1, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        
        msk=cls_92(models_92_2,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_92_cls_2, output_filename_1)
        output_path_2 = os.path.join(mod_92_cls_2, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        
        #cls 154
        
        msk=cls_154(models_154_0,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_154_cls_0, output_filename_1)
        output_path_2 = os.path.join(mod_154_cls_0, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        
        msk=cls_154(models_154_1,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_154_cls_1, output_filename_1)
        output_path_2 = os.path.join(mod_154_cls_1, output_filename_2)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

        msk=cls_154(models_154_2,image_pre,image_post)
        output_filename_1 = f"{os.path.basename(folname1)}_part1.png"
        output_filename_2 = f"{os.path.basename(folname1)}_part2.png"
        output_path_1 = os.path.join(mod_154_cls_2, output_filename_1)
        output_path_2 = os.path.join(mod_154_cls_2, output_filename_2)
        print("154CLS 2 AT",output_path_1)
        cv2.imwrite(output_path_1, msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        cv2.imwrite(output_path_2, msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])
        reda=predict_level_ofdamage()
        print(reda)
        hada=["no damage","minor damage","major damage","destroyed"]
        if reda>1:
            type=predict_image_class(post_file_path, swin_model, classes)
        else:
            type=None
        print(type)
        for folder_name in folder_names:
            folder_path = os.path.join(base_dir, folder_name)
            
            if os.path.exists(folder_path):
                # Loop through the contents of the folder and delete them
                for filename in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, filename)
                    try:
                        if os.path.isdir(file_path):
                            shutil.rmtree(file_path)  # Remove directories and their contents
                        else:
                            os.remove(file_path)  # Remove files
                        print(f"Deleted: {file_path}")
                    except Exception as e:
                        print(f"Error deleting {file_path}: {e}")
            else:
                print(f"Folder does not exist: {folder_path}")
        
        
        return {"type": type, "level": hada[reda]}

    except Exception as e:
        return {"error": str(e)}


    # finally:
    #     # Clean up the uploaded file
    #     file.file.close()


 

/kaggle/working/modeltest
=> loading checkpoint '/kaggle/working/modeltest/weights/res34_loc_0_1_best'
=> loading checkpoint '/kaggle/working/modeltest/weights/res34_loc_1_1_best'
=> loading checkpoint '/kaggle/working/modeltest/weights/res34_loc_2_1_best'
=> loading checkpoint '/kaggle/working/modeltest/weights/res50_loc_0_tuned_best'
loaded checkpoint '/kaggle/working/modeltest/weights/res50_loc_0_tuned_best' (epoch 2, best_score 0.8791586915473232)
=> loading checkpoint '/kaggle/working/modeltest/weights/res50_loc_1_tuned_best'
loaded checkpoint '/kaggle/working/modeltest/weights/res50_loc_1_tuned_best' (epoch 11, best_score 0.8668157776345657)
=> loading checkpoint '/kaggle/working/modeltest/weights/res50_loc_2_tuned_best'
loaded checkpoint '/kaggle/working/modeltest/weights/res50_loc_2_tuned_best' (epoch 12, best_score 0.872924060983002)
=> loading checkpoint '/kaggle/working/modeltest/weights/dpn92_loc_0_tuned_best'
loaded checkpoint '/kaggle/working/modeltest/weights/dpn92_loc_0

In [9]:
!ngrok config add-authtoken 2AJIi96LQAsJkU7nvb3xDCYQB4E_89fGp9rnUthkzsX7C4JKi

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
ngrok_tunnel = ngrok.connect(8000,"http")



print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app, port=8000)

Public URL: https://7ef7-34-151-77-65.ngrok-free.app


INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     105.71.18.165:0 - "GET / HTTP/1.1" 200 OK
Pre Image loaded successfully: /kaggle/working/pre_file/mexico-earthquake_00000074_pre_disaster.png, size: (1024, 1024)
Post Image loaded successfully: /kaggle/working/post_file/mexico-earthquake_00000074_post_disaster.png, size: (1024, 1024)
Processed image saved at: /kaggle/working/pred34_loc/mexico-earthquake_00000074_pre_disaster_part1.png
Processed image saved at: /kaggle/working/pred50_loc_tuned/mexico-earthquake_00000074_pre_disaster_part1.png
Processed image saved at: /kaggle/working/pred154_loc/mexico-earthquake_00000074_pre_disaster_part1.png
154CLS 2 AT /kaggle/working/se154cls_2_tuned/mexico-earthquake_00000074_pre_disaster_part1.png


100%|██████████| 2/2 [00:00<00:00, 29746.84it/s]


['mexico-earthquake_00000074_pre_disaster_part1.png']


...

100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


files mexico-earthquake_00000173_pre_disaster_part1.png


100%|██████████| 2/2 [00:00<00:00, 278.10it/s]

/kaggle/working/submission/mexico-earthquake_00000173_damage_disaster_prediction.png
/kaggle/working/submission/mexico-earthquake_00000173_localization_disaster_prediction.png
1
None
Deleted: /kaggle/working/se154cls_1_tuned/mexico-earthquake_00000173_pre_disaster_part1.png
Deleted: /kaggle/working/se154cls_1_tuned/mexico-earthquake_00000173_pre_disaster_part2.png
Deleted: /kaggle/working/dpn92cls_cce_0_tuned/mexico-earthquake_00000173_pre_disaster_part1.png
Deleted: /kaggle/working/dpn92cls_cce_0_tuned/mexico-earthquake_00000173_pre_disaster_part2.png
Deleted: /kaggle/working/dpn92cls_cce_1_tuned/mexico-earthquake_00000173_pre_disaster_part1.png
Deleted: /kaggle/working/dpn92cls_cce_1_tuned/mexico-earthquake_00000173_pre_disaster_part2.png
Deleted: /kaggle/working/dpn92cls_cce_2_tuned/mexico-earthquake_00000173_pre_disaster_part1.png
Deleted: /kaggle/working/dpn92cls_cce_2_tuned/mexico-earthquake_00000173_pre_disaster_part2.png
Deleted: /kaggle/working/pred34_loc/mexico-earthquake_000

RuntimeError: Event loop stopped before Future completed.

In [11]:
import os

# List of folder names to create
folder_names = [
    "se154cls_1_tuned",
    "dpn92cls_cce_0_tuned",
    "dpn92cls_cce_1_tuned",
    "dpn92cls_cce_2_tuned",
    "pred34_loc",
    "pred50_loc_tuned",
    "pred92_loc_tuned",
    "pred154_loc",
    "res34cls2_1_tuned",
    "res34cls2_2_tuned",
    "res50cls_cce_0_tuned",
    "res50cls_cce_1_tuned",
    "res50cls_cce_2_tuned",
    "se154cls_0_tuned",
    "se154cls_2_tuned",
    "res34cls2_0_tuned",
    "pre_file",
    "post_file",
    "submission"
]

# Base directory where folders will be created
# base_dir = "/kaggle/working"

# # Loop through the folder names and create each one
# for folder_name in folder_names:
#     folder_path = os.path.join(base_dir, folder_name)
#     os.makedirs(folder_path, exist_ok=True)
#     print(f"Created folder: {folder_path}")
for folder_name in folder_names:
    folder_path = os.path.join(base_dir, folder_name)
    
    if os.path.exists(folder_path):
        # Loop through the contents of the folder and delete them
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove directories and their contents
                else:
                    os.remove(file_path)  # Remove files
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
    else:
        print(f"Folder does not exist: {folder_path}")
print("All folders created successfully!")


Deleted: /kaggle/working/se154cls_1_tuned/joplin-tornado_00000147_pre_disaster_part2.png
Deleted: /kaggle/working/se154cls_1_tuned/joplin-tornado_00000000_pre_disaster_part2.png
Deleted: /kaggle/working/se154cls_1_tuned/hurricane-florence_00000000_pre_disaster_part1.png
Deleted: /kaggle/working/se154cls_1_tuned/guatemala-volcano_00000000_pre_disaster_part2.png
Deleted: /kaggle/working/se154cls_1_tuned/guatemala-volcano_00000000_pre_disaster_part1.png
Deleted: /kaggle/working/se154cls_1_tuned/hurricane-florence_00000000_pre_disaster_part2.png
Deleted: /kaggle/working/se154cls_1_tuned/joplin-tornado_00000000_pre_disaster_part1.png
Deleted: /kaggle/working/se154cls_1_tuned/joplin-tornado_00000147_pre_disaster_part1.png
Deleted: /kaggle/working/dpn92cls_cce_0_tuned/joplin-tornado_00000147_pre_disaster_part2.png
Deleted: /kaggle/working/dpn92cls_cce_0_tuned/joplin-tornado_00000000_pre_disaster_part2.png
Deleted: /kaggle/working/dpn92cls_cce_0_tuned/hurricane-florence_00000000_pre_disaster_p